In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
import pickle

In [2]:
df = pd.read_csv("../data/processed/dataset_processed.csv")
df.head()

,average_cyclomatic_complexity,average_methods_per_class,avg_line_length,boolean_expression_avg_terms,call_graph_density,classes,classes_with_inheritance,comment_code_mismatch_score,comment_lines,comment_percentage,...,y_GodClass,y_LargeParameterList,y_LazyClass,y_LongMethod,y_MisleadingComments,y_PoorDocumentation,y_SpaghettiCode,y_UnstableModule,y_UntestedCode,y_any_smell
0,0.360756,0.0,0.394464,0.0,0.333333,0.00000,0.000,0.0,0.125000,0.167,...,0,0,0,0,0,1,0,0,1,1
1,0.166203,0.0,0.369089,0.0,0.000000,0.00000,0.000,0.0,0.017857,0.095,...,0,1,0,0,0,1,0,0,0,1
2,0.110617,0.0,0.242215,0.0,0.000000,0.04878,0.050,0.0,0.000000,0.000,...,0,0,1,0,0,1,0,0,0,1
3,0.110617,0.0,0.226067,0.0,0.000000,0.02439,0.025,0.0,0.000000,0.000,...,0,0,1,0,0,1,0,0,0,1
4,0.110617,0.0,0.222607,0.0,0.000000,0.02439,0.025,0.0,0.000000,0.000,...,0,0,1,0,0,1,0,0,0,1


In [ ]:
X = df.iloc[:, :60].drop(columns=['file_path'])
Y = df.iloc[:, 60:]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

In [4]:
base_model = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.05,
    max_iter=200,
    min_samples_leaf=20,
    l2_regularization=0.1,
    random_state=42
)

model = MultiOutputRegressor(base_model)

model.fit(X_train, Y_train)

ValueError: could not convert string to float: 'Python\\file_transfer\\receive_file.py'

In [ ]:
Y_pred = model.predict(X_test)

for i in range(Y.shape[1]):
    print(f"\n=== Output {i} ===")
    print(classification_report(Y_test[:, i], Y_pred[:, i]))

In [ ]:
with open('../models/histogram_gb.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Histogram Gradient Boosting model saved successfully as pickle!")